 # Wpływ pandemi COVID-19 na zmianę jakości powietrza w UK #

* * * 

## 1. Wstęp
Celem niniejszego badania jest ocena wpływu pandemii COVID-19 na zmiany jakości powietrza na obszarze Wielkiej Brytanii. Punktem wyjścia dla analizy była wspólna hipoteza badawcza członków zespołu, zgodnie z którą ograniczenia mobilności społecznej, wymuszone przez wprowadzone lockdowny oraz restrykcje sanitarne, mogły znacząco wpłynąć na poziom zanieczyszczeń powietrza w analizowanym okresie.

Podstawowym źródłem danych wykorzystanym w badaniu był zbiór [Hourly Air Quality Data from the UK DEFRA AURN network for the years 2015–2023](https://www.kaggle.com/datasets/airqualityanthony/uk-defra-aurn-air-quality-data-2015-2023), zawierający szczegółowe, godzinowe dane<br /> 
o jakości powietrza pozyskane z Automatycznej Sieci Monitorującej (Automatic Urban and Rural Network – AURN). Dane obejmują pomiary dla kluczowych zanieczyszczeń atmosferycznych, takich jak: tlenek węgla (CO), tlenki azotu (NOₓ), dwutlenek azotu (NO₂), tlenek azotu (NO), ozon (O₃) oraz dwutlenek siarki (SO₂). Każdy pomiar został dodatkowo wzbogacony o dane meteorologiczne oraz współrzędne geoprzestrzenne punktów pomiarowych, co umożliwia precyzyjne przypisanie wyników do konkretnych lokalizacji i warunków atmosferycznych.

Na potrzeby badania ocena jakość powietrza określana na podstawie stężeń CO, NOₓ, NO₂, NO, O₃ i SO₂ została określona za pomocą indeksu jakości powietrza (AQI – Air Quality Index).<br />
W zależności od wartości AQI przypisuje się jedną z kategorii, np.:

| AQI      | Kategoria (PL)                              | Odpowiednik US-EPA / WHO       |
| -------- | ------------------------------------------- | ------------------------------ |
| <span style="color: black; background-color: green;"> 0–50     | Bardzo dobra (zielony)                      | Good                           |
| <span style="color: black; background-color: yellow;"> 51–100   | Dobra / Umiarkowana                         | Moderate                       |
| <span style="color: black; background-color: orange;"> 101–150  | Umiarkowana / Uciążliwa dla wrażliwych grup | Unhealthy for Sensitive Groups |
| <span style="color: black; background-color: red;"> 151–200+ | Zła, bardzo zła, ekstremalna                | Unhealthy–Hazardous            |


Aby zbadać potencjalny związek pomiędzy skalą rozprzestrzeniania się wirusa SARS-CoV-2 a jakością powietrza – pośrednio, poprzez wpływ lockdownów na aktywność społeczną i przemysłową 
<br /> – w analizie uwzględniono również dane ze strony [UKHSA data dashboard](hhttps://ukhsa-dashboard.data.gov.uk/). Wykorzystano plik ltla_newCasesBySpecimenDate. Zawiera on informacje o liczbie zakażeń w poszczególnych miastach Wielkiej Brytanii w okresie od marca 2020 do stycznia 2022 roku.

Dwa zestawy danych zostały zintegrowane na podstawie daty oraz lokalizacji (region geograficzny), przy czym dane o jakości powietrza zostały agregowane do poziomu dziennego w celu umożliwienia spójnego połączenia z dobowymi danymi dotyczącymi liczby zakażeń. Lokalizacje punktów pomiarowych zostały przypisane do odpowiadających im miast, zgodnie z systemem klasyfikacji przestrzennej stosowanym w pliku  ltla_newCasesBySpecimenDate.

>Ze względu na bardzo dużą objętość danych oraz ograniczenia przetwarzania i integracji informacji geoprzestrzennych, a także w celu zwiększenia dokładności analiz i możliwości wizualizacji wyników, podjęto decyzję o zawężeniu badanego obszaru wyłącznie do miasta Londyn. Londyn, jako największa aglomeracja w Wielkiej Brytanii, charakteryzuje się dobrze rozwiniętą siecią pomiarową (AURN). Umożliwia to przeprowadzenie rzetelnej analizy bez kompromisu względem jakości danych.

Tak przygotowany zbiór umożliwia równoległą analizę zmienności poziomu zanieczyszczeń oraz przebiegu fal pandemicznych w ujęciu czasowym, z zachowaniem pełnej spójności danych dla jednego, dobrze udokumentowanego obszaru metropolitalnego.

Zintegrowany zbiór danych pozwala na weryfikację następujących pytań badawczych:

* Czy pandemia COVID-19 miała istotny wpływ na jakość powietrza w Wielkiej Brytanii?

* Jakie zmiany w poziomach poszczególnych zanieczyszczeń atmosferycznych można zaobserwować w okresie przed i w trakcie pandemii?

* Czy różnice te są skorelowane z intensywnością poszczególnych fal zakażeń oraz wprowadzanymi ograniczeniami społecznymi?

* W jakim stopniu efekty pandemii były trwałe lub przejściowe?

Dla celów porównawczych okres pandemii został podzielony na sześć głównych faz, zgodnych z kalendarzem epidemiologicznym:
1. Początek epidemii i globalna eskalacja (grudzień 2019 – marzec 2020)

1. Pierwsza fala i lockdowny (marzec – czerwiec 2020)

1. Okres letni – częściowe poluzowania (lipiec – wrzesień 2020)

1. Druga fala pandemii (październik 2020 – styczeń 2021)

1. Początek kampanii szczepień (grudzień 2020 – marzec 2021)

1. Dominacja wariantu Delta i kolejne fale (kwiecień – grudzień 2021)



## 2. Czyszczenie i porządkowanie danych

In [ ]:
#import bibliotek

import kagglehub as kg
import pandas as pd

In [ ]:
# import danych o jakości powietrza

path = kg.dataset_download("airqualityanthony/uk-defra-aurn-air-quality-data-2015-2023")

In [ ]:
# wczytanie danych do ramki danych pandas i obróbka danych
df_air = pd.read_csv(path + "/AURN_2015_2023.csv")
#pokaz wszystkie kolumny
pd.set_option('display.max_columns', None)
df_air

Podczas weryfikacji danych na tym etapie analizy stwierdzono występowanie znacznej liczby braków (wartości typu NaN) w rejestrach pochodzących z różnych lokalizacji. W wielu przypadkach skala brakujących danych przekraczała poziom, który pozwalałby na ich wiarygodne uzupełnienie bez ryzyka zniekształcenia wyników. W związku z tym zdecydowano się na selekcję danych według kryterium kompletności.

W konsekwencji dalsza analiza została zawężona do obszaru Londynu, gdzie dostępność i jakość danych umożliwiały przeprowadzenie rzetelnej oceny zmian jakości powietrza. Analiza nie obejmuje więc całego obszaru Wielkiej Brytanii.

In [ ]:
#ramka danych dla site zawierające w nazwie London
df_air_London = df_air[df_air['site'].str.contains("London")]
# wydrukuj liczbę wierszy
print("Liczba wierszy po filtrowaniu: ", df_air_London.shape[0])

df_air_London

In [ ]:
# Liczba braków w każdej kolumnie dla site zawierającej London
print("Liczba braków w każdej kolumnie dla site zawierającej London:")
print(df_air_London[df_air_London['site'].str.contains('London', na=False)].isna().sum())

## Wizulizacja braków dla wszystkich, londyńskich stacji monitorujących.   

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
# Lista unikalnych lokalizacji z brakami danych
sites = df_air_London['site'].unique()
n = len(sites)

# Fale pandemii jako zakresy dat i kolory
pandemic_phases = [
    ("Początek epidemii i globalna eskalacja", '2019-12-01', '2020-03-01', 'lightgrey'),
    ("Pierwsza fala i lockdowny", '2020-03-01', '2020-06-30', 'lightcoral'),
    ("Okres letni – częściowe poluzowania", '2020-07-01', '2020-09-30', 'lightgreen'),
    ("Druga fala pandemii", '2020-10-01', '2021-01-31', 'lightskyblue'),
    ("Początek kampanii szczepień", '2020-12-01', '2021-03-31', 'khaki'),
    ("Dominacja wariantu Delta i kolejne fale", '2021-04-01', '2021-12-31', 'plum')
]

# Utwórz osobne wykresy dla każdej lokalizacji (bez grupowania po site_type)
fig, axes = plt.subplots(n, 1, figsize=(12, 3 * n), sharex=True)
if n == 1:
    axes = [axes]

for ax, site in zip(axes, sites):
    df_site = df_air_London[df_air_London['site'] == site]
    dates = df_site['date'].astype('datetime64[ns]')
    nox_nan = df_site['nox'].isna()
    no2_nan = df_site['no2'].isna()
    no_nan = df_site['no'].isna()
    co_nan = df_site['co'].isna()
    so2_nan = df_site['so2'].isna()
    o3_nan = df_site['o3'].isna()
    pm10_nan = df_site['pm10'].isna()
    pm25_nan = df_site['pm2.5'].isna()

    ax.scatter(dates[nox_nan], ['nox']*nox_nan.sum(), color='blue', label='nox NaN', marker='|')
    ax.scatter(dates[no2_nan], ['no2']*no2_nan.sum(), color='green', label='no2 NaN', marker='|')
    ax.scatter(dates[no_nan], ['no']*no_nan.sum(), color='red', label='no NaN', marker='|')
    ax.scatter(dates[co_nan], ['co']*co_nan.sum(), color='orange', label='co NaN', marker='|')
    ax.scatter(dates[so2_nan], ['so2']*so2_nan.sum(), color='purple', label='so2 NaN', marker='|')
    ax.scatter(dates[o3_nan], ['o3']*o3_nan.sum(), color='brown', label='o3 NaN', marker='|')
    ax.scatter(dates[pm10_nan], ['pm10']*pm10_nan.sum(), color='pink', label='pm10 NaN', marker='|')
    ax.scatter(dates[pm25_nan], ['pm2.5']*pm25_nan.sum(), color='cyan', label='pm2.5 NaN', marker='|')     

    # Dodanie zakresów pandemicznych jako tła
    for phase, start, end, color in pandemic_phases:
        ax.axvspan(pd.to_datetime(start), pd.to_datetime(end), color=color, alpha=0.3)

    ax.set_ylabel('Zanieczyszczenie')
    ax.set_title(f'Lokalizacja: {site}')
    ax.grid(True)

# Dodanie legendy pandemicznych faz pod wykresami
legend_patches = [mpatches.Patch(color=color, label=phase) for phase, _, _, color in pandemic_phases]
plt.legend(handles=legend_patches, loc='upper center', bbox_to_anchor=(0.5, -0.4), ncol=2)

plt.xlabel('Rok')
plt.suptitle('Braki danych (NaN) dla no, nox, no2, co, so2, o3\nFale pandemii oznaczone kolorami (tylko lokalizacje zawierające "London")')
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


Na obecnym etapie analizy podjęto decyzję o ograniczeniu liczby badanych rodzajów zanieczyszczeń. Głównym powodem tej decyzji była zbyt duża liczba brakujących danych (tzw. missing values) dla części zmiennych. Uwzględnienie ich w dalszych etapach mogłoby istotnie wpłynąć na rzetelność i wiarygodność wyników analizy statystycznej. Dlatego do dalszych obliczeń wybrano wyłącznie te zmienne, które charakteryzowały się wystarczającą kompletnością danych.

In [ ]:
#usuwnie kolumn z ramki danych df_air_London
df_air_London = df_air_London.drop(columns=['latitude', 'longitude', 'co','so2','nv10','nv2.5','v10','v2.5','ws', 'wd','air_temp','nox','code'])
# zapisanie df_air_London do pliku csv
df_air_London.to_csv("Dane/df_air_London.csv", index=False)

In [ ]:
df_air_London = pd.read_csv("Dane/df_air_London.csv")
df_air_London

Na potrzeby analizy przygotowano zbiór danych zawierający informacje o liczbie zachorowań na COVID-19.

In [ ]:
#ramka danych z ltla_newCasesBySpecimenDate
df_ltla = pd.read_csv("Dane/ltla_newCasesBySpecimenDate.csv")

#usuwanie kolumn z ramki danych df_ltla
#df_ltla = df_ltla.drop(columns=['metric','metric_name'])

#dane dla area_name zawierającej 'London'
df_ltla = df_ltla[df_ltla['area_name'].str.contains("London")]
# wydrukuj liczbę wierszy
df_ltla.count()

# zapisanie df_ltla do pliku csv
df_ltla.to_csv("Dane/ltla_newCasesBySpecimenDate.csv", index=False)

W celu zwiększenia kompletności zbioru danych, brakujące wartości zostały uzupełnione przy użyciu interpolacji dwustronnej. Metoda ta pozwala na oszacowanie brakujących obserwacji na podstawie wartości sąsiednich, zarówno poprzedzających, jak i następujących po danym braku, co sprzyja zachowaniu ciągłości i spójności danych w czasie.

Zastosowanie interpolacji dwustronnej umożliwiło ograniczenie liczby braków bez wprowadzania istotnych zniekształceń w strukturze danych, a tym samym pozwoliło na uwzględnienie większej liczby obserwacji w dalszej analizie.

In [ ]:
import pandas as pd
import numpy as np

# Wczytanie danych
df_air_London = pd.read_csv("Dane/df_air_London.csv")
df_ltla = pd.read_csv("Dane/ltla_newCasesBySpecimenDate.csv")
# Upewnij się, że 'date' to typ daty
df_air_London['date'] = pd.to_datetime(df_air_London['date'])

# Wybierz kolumny z brakami do uzupełnienia
cols = ['date','no2', 'o3', 'pm2.5', 'pm10']
df_air_London = df_air_London[cols]

# Interpolacja (dwustronna: forward + backward)
df_air_London.interpolate(method='linear', limit_direction='both', inplace=True)

print(df_air_London.isna().sum())  # Sprawdź, czy są nadal braki

Zbiór danych zawierał liczne punkty pomiarowe jakości powietrza, rozmieszczone w różnych lokalizacjach. Aby możliwe było przeprowadzenie analizy porównawczej z danymi dotyczącymi liczby zachorowań, konieczne było ujednolicenie struktury danych w czasie. W tym celu wyniki pomiarów zostały zagregowane w ramach poszczególnych dat.

Dla każdego rodzaju zanieczyszczenia przyjęto wartość maksymalną spośród wszystkich lokalizacji z danego dnia. Takie podejście pozwalało uchwycić potencjalnie najwyższy poziom narażenia populacji na zanieczyszczenie w danym okresie, co uznano za istotne z punktu widzenia analizy.

In [ ]:
#grupowanie danych po dacie i wybranie maksymalnej wartości dla każdej daty 
df_air_London_grouped = df_air_London.groupby(df_air_London['date'].dt.date).aggregate({
    'no2': 'max',
    'o3': 'max',
    'pm2.5': 'max',     
    'pm10': 'max'
}).reset_index()
df_air_London_grouped 

Do oceny jakości powietrza w analizie zastosowano wskaźnik AQI (Air Quality Index), który umożliwia przekształcenie danych pomiarowych w ujednoliconą skalę oceny poziomu zanieczyszczeń.

## Wyznaczanie wskaźnika AQI
Dla każdego zanieczyszczenia wyliczany jest wskażnik **IAQI**
<br />

$$\ I = \frac {I_{high} - I_{low}}{C_{high} - C_{low}}\left( C - C_{low} \right) + I_{low} $$
<br />
Gdzie: <br />
 I – Air Quality indeks, <br />
 C – Zmierzone stężenie, <br />
 C<sub>low</sub> , C<sub>high</sub> – górna i dolna granica przedziału, w którym mieści się C,<br />
 I<sub>low</sub> , I<sub>high</sub> – wartości indeksu jakości powietrza odpowiadające tym granicom.


Ze względu na specyfikę wskaźnika AQI konieczne było wcześniejsze dostosowanie jednostek miar. Wartości poszczególnych zanieczyszczeń, początkowo wyrażone w metrach sześciennych, zostały przeliczone na odpowiednie jednostki wymagane dla danego rodzaju substancji (np. µg/m³ lub ppm), zgodnie z obowiązującymi normami wyznaczania AQI. Działanie to było niezbędne dla zapewnienia poprawności interpretacji wartości indeksu.


In [ ]:
# przeliczenie wartosci z jedniostki µg/m³ na jednostki wymagane przez bibliotekę aqi. o3 na ppm, no2 na ppb
# Dla tlenku azotu (NO₂) w warunkach standardowych (25°C i 1 atm), współczynnik konwersji z ppb na µg/m³ wynosi około 1.91. Oznacza to, że 1 ppb NO₂ odpowiada około 1.91 µg/m³ NO₂. 
#df_air_London_grouped['no2'] = df_air_London_grouped['no2'] / 1.91 
# Aby przeliczyć ozon (O3) z µg/m³ na ppm, należy użyć współczynnika konwersji. 1 ppm ozonu odpowiada w przybliżeniu 2140 µg/m³. 
df_air_London_grouped['o3'] = df_air_London_grouped['o3'] / 2140

In [ ]:
df_air_London_grouped

In [ ]:
# calculate AQI for each row in df_air_London_grouped aqi.to_iaqi
df_air_London_grouped['AQI_PM25'] = df_air_London_grouped.apply(
    lambda row: aqi.to_iaqi(aqi.POLLUTANT_PM25, row['pm2.5'], algo=aqi.ALGO_EPA), axis=1
)
df_air_London_grouped['AQI_PM10'] = df_air_London_grouped.apply(
    lambda row: aqi.to_iaqi(aqi.POLLUTANT_PM25, row['pm10'], algo=aqi.ALGO_EPA), axis=1
)
df_air_London_grouped['AQI_NO2'] = df_air_London_grouped.apply(
    lambda row: aqi.to_iaqi(aqi.POLLUTANT_NO2_1H, row['no2'], algo=aqi.ALGO_EPA), axis=1
)
df_air_London_grouped['AQI_O3'] = df_air_London_grouped.apply(
    lambda row: aqi.to_iaqi(aqi.POLLUTANT_O3_8H, row['o3'], algo=aqi.ALGO_EPA), axis=1

In [ ]:
# aqi biblioteki i liczenie AQI
# pip install python-aqi
 
# https://pypi.org/project/python-aqi/ - link do dokumentacji
# https://github.com/hrbonz/python-aqi/tree/master/aqi - github projektu
                  


# calculate AQI for each row in df_air_London_grouped aqi.to_iaqi
df_air_London_grouped['AQI_PM25'] = df_air_London_grouped.apply(
    lambda row: aqi.to_iaqi(aqi.POLLUTANT_PM25, row['pm2.5'], algo=aqi.ALGO_EPA), axis=1
)
df_air_London_grouped['AQI_PM10'] = df_air_London_grouped.apply(
    lambda row: aqi.to_iaqi(aqi.POLLUTANT_PM25, row['pm10'], algo=aqi.ALGO_EPA), axis=1
)
df_air_London_grouped['AQI_NO2'] = df_air_London_grouped.apply(
    lambda row: aqi.to_iaqi(aqi.POLLUTANT_NO2_1H, row['no2'], algo=aqi.ALGO_EPA), axis=1
)
df_air_London_grouped['AQI_O3'] = df_air_London_grouped.apply(
    lambda row: aqi.to_iaqi(aqi.POLLUTANT_O3_8H, row['o3'], algo=aqi.ALGO_EPA), axis=1
)  





# utworzenie ramki danych df_air_YK11 z pliku df_air_YK11.csv
#df_air_YK11 = pd.read_csv("C:\GITHUB\SAD-1\Dane/df_air_YK11.csv")
 
# For each row in df_air_YK11, calculate AQI and add it to a new column
"""def calc_aqi(row):
    try:
        if pd.notna(row['o3']) and pd.notna(row['no2']) and pd.notna(row['pm2.5']) and pd.notna(row['pm10']):
            return aqi.to_aqi([(aqi.POLLUTANT_O3_1H, row['o3']), (aqi.POLLUTANT_NO2_1H, row['no2']), (aqi.POLLUTANT_PM25, row['pm2.5']), (aqi.POLLUTANT_PM10, row['pm10'])], algo=aqi.ALGO_EPA)
        else:
            return None
    except Exception:
        return None
df_air_London_grouped['AQI_day_all'] = df_air_London_grouped.apply(calc_aqi, axis=1)"""
 
# dane z df_air_London z kolumną AQI nie zawierającą wartości None
#print(df_air_London[df_air_London['AQI'].notna()])
 
df_air_London_grouped.count()
# wyswietlanie wierszy z wyliczonymi AQI
print(df_air_London_grouped[df_air_London_grouped['AQI_PM25'].notna()])

# dodaj kolumne z maksimum z AQI składników 
df_air_London_grouped['AQI_day_max'] = df_air_London_grouped[['AQI_NO2', 'AQI_O3', 'AQI_PM25', 'AQI_PM10']].max(axis=1)



# maximum AQI_PM25
max_aqi_pm25 = df_air_London_grouped['AQI_PM25'].max()
print(f'Maximum AQI_PM25: {max_aqi_pm25}')
#pokazanie daty maksymalnej wartości dla pm2.5
max_pm25_date = df_air_London_grouped.loc[df_air_London_grouped['AQI_PM25'].idxmax(), 'date']
print(f'Data maksymalnej wartości AQI dla PM2.5: {max_pm25_date}')  
#zaprezentuj cały wiersz z danym i dla maksymalnej wartości pm2.5
max_pm25_row = df_air_London_grouped[df_air_London_grouped['AQI_PM25'] == max_aqi_pm25]
print(max_pm25_row)